### 0. Helper functions

In [1]:
import numpy as np

class bin_data:
    def __init__(self, env):
        self.total_capacity = np.full(env.num_bins, 0) # Total capacity seen across all trials
        self.capacity_used = np.full(env.num_bins, 0) # Total capacity used across all trials
        self.steps_taken = 0 # Number of timesteps taken before trial terminated
        self.placed = 0 # Number of items correctly placed
        self.misplaced = 0
        self.discarded = 0
        self.data_points = 0 # Number of trials we've witnessed
    

    def log(self, env):
        self.total_capacity = np.add(self.total_capacity, env.capacity)
        self.capacity_used = np.add(self.capacity_used, env.state[:-1])
        self.steps_taken += sum(env.logs.values())
        self.placed += env.logs["placed"]
        self.misplaced += env.logs["misplaced"]
        self.discarded += env.logs["discarded"]
        self.data_points += 1
        

    def get_avg(self):
        percentages = 100 - ((self.capacity_used/self.total_capacity) * 100)
        
        return {
            "steps" : self.steps_taken / self.data_points,
            "utilization" : sum(percentages) / len(percentages),
            "accuracy" : (self.placed / (self.placed + self.misplaced)) * 100
        }
    

    def print_data(self):
        data = self.get_avg()
        print(f'Average number of steps taken: {data["steps"]}')
        print(f'Average bin utilization: {round(data["utilization"], 2)}%')
        print(f'Accuracy: {round(data["accuracy"], 2)}%')

### 1. Import dependencies

In [2]:
!pip install tensorflow==2.7.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [2]:
from BinPackingEnvironment1D import BinPacking

### 2. Create environment

In [3]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [4]:
env.action_space.sample()

3

In [5]:
env.observation_space.sample()

array([ 6, 19, 13,  8,  8,  7, 19,  7, 20,  0, 13])

### 3. Run baseline test (No ML)

In [6]:
control_data = bin_data(env)

In [7]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    control_data.log(env)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-2334
{'placed': 41, 'misplaced': 225, 'discarded': 25}
Episode:2 Score:-2094
{'placed': 41, 'misplaced': 198, 'discarded': 31}
Episode:3 Score:-2656
{'placed': 44, 'misplaced': 251, 'discarded': 38}
Episode:4 Score:-3268
{'placed': 42, 'misplaced': 309, 'discarded': 44}
Episode:5 Score:-1381
{'placed': 44, 'misplaced': 135, 'discarded': 15}
Episode:6 Score:-4921
{'placed': 44, 'misplaced': 476, 'discarded': 41}
Episode:7 Score:-5680
{'placed': 40, 'misplaced': 544, 'discarded': 56}
Episode:8 Score:-2617
{'placed': 43, 'misplaced': 255, 'discarded': 22}
Episode:9 Score:-5184
{'placed': 41, 'misplaced': 496, 'discarded': 53}
Episode:10 Score:-4342
{'placed': 43, 'misplaced': 415, 'discarded': 47}


### 4. Train an RL Model

In [8]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [9]:
# Will throw an error if these don't exist
log_path = os.path.join('Training', 'Logs')

In [10]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [11]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
model.learn(total_timesteps=100000)

Logging to Training\Logs\PPO_64
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 381       |
|    ep_rew_mean     | -3.14e+03 |
| time/              |           |
|    fps             | 239       |
|    iterations      | 1         |
|    time_elapsed    | 8         |
|    total_timesteps | 2048      |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 326          |
|    ep_rew_mean          | -2.61e+03    |
| time/                   |              |
|    fps                  | 349          |
|    iterations           | 2            |
|    time_elapsed         | 11           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0105625205 |
|    clip_fraction        | 0.0458       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.39        |
|    explained_v

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 259         |
|    ep_rew_mean          | -1.78e+03   |
| time/                   |             |
|    fps                  | 574         |
|    iterations           | 11          |
|    time_elapsed         | 39          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.007876963 |
|    clip_fraction        | 0.039       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.98       |
|    explained_variance   | -8.05e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 3.75e+03    |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0117     |
|    value_loss           | 7.81e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 255   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 186         |
|    ep_rew_mean          | -882        |
| time/                   |             |
|    fps                  | 582         |
|    iterations           | 21          |
|    time_elapsed         | 73          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.008258713 |
|    clip_fraction        | 0.0705      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.53       |
|    explained_variance   | 2.17e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 1.49e+03    |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.0148     |
|    value_loss           | 3.04e+03    |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 177 

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 122          |
|    ep_rew_mean          | -414         |
| time/                   |              |
|    fps                  | 578          |
|    iterations           | 31           |
|    time_elapsed         | 109          |
|    total_timesteps      | 63488        |
| train/                  |              |
|    approx_kl            | 0.0077088224 |
|    clip_fraction        | 0.0695       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.41        |
|    explained_variance   | 0.53         |
|    learning_rate        | 0.0003       |
|    loss                 | 635          |
|    n_updates            | 300          |
|    policy_gradient_loss | -0.0153      |
|    value_loss           | 1.11e+03     |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 102         |
|    ep_rew_mean          | -271        |
| time/                   |             |
|    fps                  | 576         |
|    iterations           | 41          |
|    time_elapsed         | 145         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.008551028 |
|    clip_fraction        | 0.0742      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.26       |
|    explained_variance   | 0.564       |
|    learning_rate        | 0.0003      |
|    loss                 | 399         |
|    n_updates            | 400         |
|    policy_gradient_loss | -0.0152     |
|    value_loss           | 890         |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 99.8

### 5. Save model

In [14]:
#PPO_Path = os.path.join('Training', 'Saved Models', 'Constant_PPO_Model_Discard_Penalty')

In [15]:
#model.save(PPO_Path)

### 6. Load model

In [16]:
#model = PPO.load(PPO_Path, env=env)

### 7. Test model

In [15]:
real_data = bin_data(env)
env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

In [16]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    obs = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    real_data.log(env)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-171
{'placed': 44, 'misplaced': 4, 'discarded': 35}
Episode:2 Score:-357
{'placed': 43, 'misplaced': 17, 'discarded': 46}
Episode:3 Score:-193
{'placed': 52, 'misplaced': 13, 'discarded': 23}
Episode:4 Score:-143
{'placed': 47, 'misplaced': 9, 'discarded': 20}
Episode:5 Score:-387
{'placed': 48, 'misplaced': 11, 'discarded': 65}
Episode:6 Score:-325
{'placed': 45, 'misplaced': 5, 'discarded': 64}
Episode:7 Score:-131
{'placed': 44, 'misplaced': 7, 'discarded': 21}
Episode:8 Score:-222
{'placed': 53, 'misplaced': 5, 'discarded': 45}
Episode:9 Score:-287
{'placed': 53, 'misplaced': 2, 'discarded': 64}
Episode:10 Score:-236
{'placed': 49, 'misplaced': 11, 'discarded': 35}


In [17]:
control_data.print_data()

Average number of steps taken: 409.9
Average bin utilization: 98.35%
Accuracy: 11.35%


In [18]:
real_data.print_data()

Average number of steps taken: 98.0
Average bin utilization: 97.7%
Accuracy: 85.05%
